<a href="https://colab.research.google.com/github/azzindani/03_LLM_Fine_Tune/blob/main/Llama3.2_3B_Instruct_Fine_Tune_PEFT_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 00 Import Modules

In [ ]:
#!pip install --upgrade transformers
!pip install -q peft
!pip install -U -q bitsandbytes
!pip install -q datasets
!pip install -q trl

In [ ]:
import os
import pathlib
import torch
import numpy as np
import textwrap

from random import randint
from itertools import zip_longest
from datetime import datetime
from datasets import load_dataset
from datasets import Dataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from trl import SFTTrainer

from transformers import (
  AutoTokenizer,
  AutoModelForCausalLM,
  AutoModelForSeq2SeqLM,
  AutoModel,
  AutoModelForSequenceClassification,
  DataCollatorForLanguageModeling,
  Trainer,
  TrainingArguments,
  pipeline,
  TextDataset,
  EvalPrediction,
  DataCollatorWithPadding,
  GenerationConfig,
  BitsAndBytesConfig,
  DataCollatorForSeq2Seq,
  TextStreamer
)

from peft import (
  LoraConfig,
  PeftModelForSequenceClassification,
  PeftModel,
  TaskType,
  AutoPeftModelForSequenceClassification,
  get_peft_model,
  prepare_model_for_kbit_training
)

if torch.cuda.is_available():
  print("GPU is available!")
else:
  print("GPU is not available.")

GPU is available!


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

## 01 Import Model

In [ ]:
#url = 'https://huggingface.co/Qwen/Qwen2.5-0.5B'
#model_name = url.split('.co/')[-1]

model_name = 'unsloth/Llama-3.2-3B-Instruct'

In [ ]:
def load_model(model_name, base = True):
    if base == True:
        model = AutoModelForCausalLM.from_pretrained(
            model_name,
            torch_dtype = torch.float16,
            trust_remote_code = True
        ).to(device)

        return model

    else:
        bnb_config = BitsAndBytesConfig(
            load_in_4bit = True,
            bnb_4bit_quant_type = 'nf4',
            bnb_4bit_compute_dtype = torch.float16,
            bnb_4bit_use_double_quant = True,
        )
        model = AutoModelForCausalLM.from_pretrained(
            model_name,
            quantization_config = bnb_config,
            trust_remote_code = True
        ).to(device)

        return model

In [ ]:
model = load_model(model_name, base = False)
model

`low_cpu_mem_usage` was None, now default to True since model is quantized.


LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 3072)
    (layers): ModuleList(
      (0-27): 28 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear4bit(in_features=3072, out_features=3072, bias=False)
          (k_proj): Linear4bit(in_features=3072, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=3072, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=3072, out_features=3072, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=3072, out_features=8192, bias=False)
          (up_proj): Linear4bit(in_features=3072, out_features=8192, bias=False)
          (down_proj): Linear4bit(in_features=8192, out_features=3072, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((3072,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((3072,), eps=1e

In [ ]:
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
trainable_percentage = (trainable_params / total_params) * 100

print('Total parameters :', total_params)
print('Trainable parameters :', trainable_params)
print('Trainable percentage: {:.2f}%'.format(trainable_percentage))

Total parameters : 1803463680
Trainable parameters : 394177536
Trainable percentage: 21.86%


## 02 Import Tokenizer

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
#tokenizer

## 03 Import Dataset

In [ ]:
#url = 'https://huggingface.co/datasets/KingNish/reasoning-base-20k'
#dataset_name = url.split('datasets/')[-1]

dataset_name = 'mlabonne/FineTome-100k'

In [ ]:
max_length = 1024

In [ ]:
dataset = load_dataset(dataset_name, split = 'train')
dataset

Dataset({
    features: ['conversations', 'source', 'score'],
    num_rows: 100000
})

In [ ]:
dataset = dataset.select(range(10000))

In [ ]:
dataset.select(range(5)).to_pandas().head()

,conversations,source,score
0,"[{'from': 'human', 'value': 'Explain what bool...",infini-instruct-top-500k,5.212621
1,"[{'from': 'human', 'value': 'Explain how recur...",infini-instruct-top-500k,5.157649
2,"[{'from': 'human', 'value': 'Explain what bool...",infini-instruct-top-500k,5.147540
3,"[{'from': 'human', 'value': 'Explain the conce...",infini-instruct-top-500k,5.053656
4,"[{'from': 'human', 'value': 'Print the reverse...",infini-instruct-top-500k,5.045648


In [ ]:
dataset[0]

{'conversations': [{'from': 'human',
   'value': 'Explain what boolean operators are, what they do, and provide examples of how they can be used in programming. Additionally, describe the concept of operator precedence and provide examples of how it affects the evaluation of boolean expressions. Discuss the difference between short-circuit evaluation and normal evaluation in boolean expressions and demonstrate their usage in code. \n\nFurthermore, add the requirement that the code must be written in a language that does not support short-circuit evaluation natively, forcing the test taker to implement their own logic for short-circuit evaluation.\n\nFinally, delve into the concept of truthiness and falsiness in programming languages, explaining how it affects the evaluation of boolean expressions. Add the constraint that the test taker must write code that handles cases where truthiness and falsiness are implemented differently across different programming languages.'},
  {'from': 'gpt

In [ ]:
features = list(dataset.features.keys())
print(features)

['conversations', 'source', 'score']


## 04 Text Formatting

In [ ]:
def transform_conversations(example):
  role_map = {
    'human' : 'user',
    'gpt' : 'assistant'
  }

  transformed_conversations = [
    {
      'role' : role_map.get(turn['from'], turn['from']),
      'content' : turn['value']
    }
    for turn in example['conversations']
  ]
  return {'conversations': transformed_conversations}

In [ ]:
formatted_dataset = dataset.map(transform_conversations, remove_columns = features)
formatted_dataset

Dataset({
    features: ['conversations'],
    num_rows: 10000
})

In [ ]:
print(formatted_dataset[0]['conversations'])

[{'content': 'Explain what boolean operators are, what they do, and provide examples of how they can be used in programming. Additionally, describe the concept of operator precedence and provide examples of how it affects the evaluation of boolean expressions. Discuss the difference between short-circuit evaluation and normal evaluation in boolean expressions and demonstrate their usage in code. \n\nFurthermore, add the requirement that the code must be written in a language that does not support short-circuit evaluation natively, forcing the test taker to implement their own logic for short-circuit evaluation.\n\nFinally, delve into the concept of truthiness and falsiness in programming languages, explaining how it affects the evaluation of boolean expressions. Add the constraint that the test taker must write code that handles cases where truthiness and falsiness are implemented differently across different programming languages.', 'role': 'user'}, {'content': 'Boolean operators are 

In [ ]:
def format_conversation(example):
  for entry in example['conversations']:
    role = entry['role']
    content = entry['content']

    if role == 'user':
      formatted_text = f"<|start_header_id|>{role}<|end_header_id|>\n\n{content}\n<|eot_id|>"
    elif role == 'assistant':
      formatted_text += f"<|start_header_id|>{role}<|end_header_id|>\n\n{content}\n<|eot_id|>"

  return {'prompt': formatted_text}

In [ ]:
formatted_dataset = formatted_dataset.map(
  format_conversation,
  remove_columns = list(formatted_dataset.features.keys())
)
formatted_dataset

Dataset({
    features: ['prompt'],
    num_rows: 10000
})

In [ ]:
print(formatted_dataset[0]['prompt'])

<|start_header_id|>user<|end_header_id|>

Explain what boolean operators are, what they do, and provide examples of how they can be used in programming. Additionally, describe the concept of operator precedence and provide examples of how it affects the evaluation of boolean expressions. Discuss the difference between short-circuit evaluation and normal evaluation in boolean expressions and demonstrate their usage in code. 

Furthermore, add the requirement that the code must be written in a language that does not support short-circuit evaluation natively, forcing the test taker to implement their own logic for short-circuit evaluation.

Finally, delve into the concept of truthiness and falsiness in programming languages, explaining how it affects the evaluation of boolean expressions. Add the constraint that the test taker must write code that handles cases where truthiness and falsiness are implemented differently across different programming languages.
<|eot_id|><|start_header_id|>a

## 05 Tokenization

In [ ]:
def tokenize_data(example, max_length = max_length):
  return tokenizer(example['prompt'], truncation = True, padding = 'max_length', max_length = max_length)

In [ ]:
tokenized_dataset = formatted_dataset.map(tokenize_data, batched = True)#, remove_columns = 'text')
tokenized_dataset

Dataset({
    features: ['prompt', 'input_ids', 'attention_mask'],
    num_rows: 10000
})

In [ ]:
print(tokenized_dataset[0]['prompt'])

<|start_header_id|>user<|end_header_id|>

Explain what boolean operators are, what they do, and provide examples of how they can be used in programming. Additionally, describe the concept of operator precedence and provide examples of how it affects the evaluation of boolean expressions. Discuss the difference between short-circuit evaluation and normal evaluation in boolean expressions and demonstrate their usage in code. 

Furthermore, add the requirement that the code must be written in a language that does not support short-circuit evaluation natively, forcing the test taker to implement their own logic for short-circuit evaluation.

Finally, delve into the concept of truthiness and falsiness in programming languages, explaining how it affects the evaluation of boolean expressions. Add the constraint that the test taker must write code that handles cases where truthiness and falsiness are implemented differently across different programming languages.
<|eot_id|><|start_header_id|>a

In [ ]:
tokenized_dataset = tokenized_dataset.train_test_split(test_size = 0.1, seed = 42)
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['prompt', 'input_ids', 'attention_mask'],
        num_rows: 9000
    })
    test: Dataset({
        features: ['prompt', 'input_ids', 'attention_mask'],
        num_rows: 1000
    })
})

In [ ]:
train_dataset = tokenized_dataset['train']
test_dataset = tokenized_dataset['test']
train_dataset

Dataset({
    features: ['prompt', 'input_ids', 'attention_mask'],
    num_rows: 9000
})

In [ ]:
train_dataset.select(range(5)).to_pandas().head()

,prompt,input_ids,attention_mask
0,<|start_header_id|>user<|end_header_id|>\n\nHo...,"[128004, 128004, 128004, 128004, 128004, 12800...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,<|start_header_id|>user<|end_header_id|>\n\nWr...,"[128004, 128004, 128004, 128004, 128004, 12800...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,<|start_header_id|>user<|end_header_id|>\n\nWh...,"[128004, 128004, 128004, 128004, 128004, 12800...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,<|start_header_id|>user<|end_header_id|>\n\nHo...,"[128004, 128004, 128004, 128004, 128004, 12800...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,<|start_header_id|>user<|end_header_id|>\n\nHo...,"[128004, 128004, 128004, 128004, 128004, 12800...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [ ]:
print(train_dataset[0]['prompt'])

<|start_header_id|>user<|end_header_id|>

How do you graph the inequality #x > -4#?
<|eot_id|><|start_header_id|>assistant<|end_header_id|>

To graph the inequality #x > -4#, follow these steps:

1. Start by plotting the boundary line: Draw a vertical line at #x = -4# on the X-axis. This line divides the X-axis into two halves.

2. Identify the direction of the inequality: Since the inequality is #x > -4#, it indicates that we are looking for all values of #x# that are greater than -4.

3. Shade the appropriate region: Shade the entire region to the right of the line at #x = -4#. This shaded area represents all the points whose X-coordinate is greater than -4.

4. Include a dashed line: Since the inequality is "greater than" (#>,# not "#≥#"), draw the line at #x = -4# as a dashed line, indicating that the points on the line itself are not included in the solution.

The graph consists of the entire half-plane to the right of the dashed vertical line at #x = -4#, excluding the line itsel

In [ ]:
print(train_dataset[0]['input_ids'])

[128004, 128004, 128004, 128004, 128004, 128004, 128004, 128004, 128004, 128004, 128004, 128004, 128004, 128004, 128004, 128004, 128004, 128004, 128004, 128004, 128004, 128004, 128004, 128004, 128004, 128004, 128004, 128004, 128004, 128004, 128004, 128004, 128004, 128004, 128004, 128004, 128004, 128004, 128004, 128004, 128004, 128004, 128004, 128004, 128004, 128004, 128004, 128004, 128004, 128004, 128004, 128004, 128004, 128004, 128004, 128004, 128004, 128004, 128004, 128004, 128004, 128004, 128004, 128004, 128004, 128004, 128004, 128004, 128004, 128004, 128004, 128004, 128004, 128004, 128004, 128004, 128004, 128004, 128004, 128004, 128004, 128004, 128004, 128004, 128004, 128004, 128004, 128004, 128004, 128004, 128004, 128004, 128004, 128004, 128004, 128004, 128004, 128004, 128004, 128004, 128004, 128004, 128004, 128004, 128004, 128004, 128004, 128004, 128004, 128004, 128004, 128004, 128004, 128004, 128004, 128004, 128004, 128004, 128004, 128004, 128004, 128004, 128004, 128004, 128004,

In [ ]:
print(train_dataset[0]['attention_mask'])

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

## 06 Data Collator Set Up

In [ ]:
#data_collator = DataCollatorWithPadding(tokenizer = tokenizer)
#data_collator = DataCollatorForSeq2Seq(tokenizer = tokenizer)
data_collator = DataCollatorForLanguageModeling(tokenizer = tokenizer, mlm = False)

## 07 Evaluation Metrics Set Up

In [ ]:
def compute_metrics(p: EvalPrediction):
  preds = np.argmax(p.predictions, axis = 1)
  precision, recall, f1, _ = precision_recall_fscore_support(
    p.label_ids,
    preds,
    average = 'weighted'
  )
  matrix = {
    'accuracy': accuracy_score(p.label_ids, preds),
    'f1': f1, 'precision': precision,
    'recall': recall
  }
  return matrix

In [ ]:
torch.cuda.empty_cache()

## 08 Set Up PEFT / LoRA / QLoRA

In [ ]:
lora_alpha = 16
lora_dropout = 0.1
lora_r = 64
target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                  "gate_proj", "up_proj", "down_proj",]
peft_config = LoraConfig(
  lora_alpha = lora_alpha,
  lora_dropout = lora_dropout,
  r = lora_r,
  bias = 'none',
  task_type = 'CAUSAL_LM',
  target_modules = target_modules,
)

In [ ]:
peft_model = get_peft_model(model, peft_config, adapter_name = 'LoRA')
peft_model.print_trainable_parameters()

trainable params: 97,255,424 || all params: 3,310,005,248 || trainable%: 2.9382


## 09 Training Model

In [ ]:
model

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 3072)
    (layers): ModuleList(
      (0-27): 28 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): lora.Linear4bit(
            (base_layer): Linear4bit(in_features=3072, out_features=3072, bias=False)
            (lora_dropout): ModuleDict(
              (LoRA): Dropout(p=0.1, inplace=False)
            )
            (lora_A): ModuleDict(
              (LoRA): Linear(in_features=3072, out_features=64, bias=False)
            )
            (lora_B): ModuleDict(
              (LoRA): Linear(in_features=64, out_features=3072, bias=False)
            )
            (lora_embedding_A): ParameterDict()
            (lora_embedding_B): ParameterDict()
            (lora_magnitude_vector): ModuleDict()
          )
          (k_proj): lora.Linear4bit(
            (base_layer): Linear4bit(in_features=3072, out_features=1024, bias=False)
            (lora_dropout): ModuleDict(
            

In [ ]:
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
trainable_percentage = (trainable_params / total_params) * 100

print('Total parameters :', total_params)
print('Trainable parameters :', trainable_params)
print('Trainable percentage: {:.2f}%'.format(trainable_percentage))

Total parameters : 1900719104
Trainable parameters : 97255424
Trainable percentage: 5.12%


In [ ]:
torch.cuda.empty_cache()

In [ ]:
save_path = './model'

batch_size = 1
max_steps = 200
training_args = TrainingArguments(
  output_dir = save_path,
  gradient_accumulation_steps = 4,
  evaluation_strategy = 'steps',
  do_eval = True,
  per_device_train_batch_size = batch_size,
  per_device_eval_batch_size = 4,
  log_level = 'debug',
  save_strategy = 'no',
  save_total_limit = 2,
  save_safetensors = False,
  fp16 = True,
  logging_steps = 20,
  learning_rate = 2e-5,
  eval_steps = 20,
  max_steps = max_steps,
  warmup_steps = 30,
  lr_scheduler_type = 'cosine',
)
training_args

/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


TrainingArguments(
_n_gpu=1,
accelerator_config={'split_batches': False, 'dispatch_batches': None, 'even_batches': True, 'use_seedable_sampler': True, 'non_blocking': False, 'gradient_accumulation_kwargs': None, 'use_configured_state': False},
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
average_tokens_across_devices=False,
batch_eval_metrics=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_persistent_workers=False,
dataloader_pin_memory=True,
dataloader_prefetch_factor=None,
ddp_backend=None,
ddp_broadcast_buffers=None,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
ddp_timeout=1800,
debug=[],
deepspeed=None,
disable_tqdm=False,
dispatch_batches=None,
do_eval=True,
do_predict=False,
do_train=False,
eval_accumulation_steps=None,
eval_delay=0,
eval_do_concat_batches=True,
eval_on_start=False,
eval_steps=20,
eval_strategy=steps,
eval_use_gather_object=F

In [ ]:
trainer = SFTTrainer(
  model = model,
  train_dataset = train_dataset,#.select(range(10000)),
  eval_dataset = test_dataset.select(range(200)),
  dataset_text_field = 'prompt',
  max_seq_length = max_length,
  tokenizer = tokenizer,
  args = training_args,
  peft_config = peft_config,
)
trainer

/opt/conda/lib/python3.10/site-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': dataset_text_field, max_seq_length. Will not be supported from version '0.13.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:300: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will ove

In [ ]:
trainer.train()

## 10 Model Evaluation

In [ ]:
evaluation_results = trainer.evaluate()
print('Evaluation Results:', evaluation_results)

## 11 Save Model

In [ ]:
save_model = trainer.model.module if hasattr(trainer.model, 'module') else trainer.model
save_model.save_pretrained(save_path)

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--unsloth--Llama-3.2-3B-Instruct/snapshots/bc836a93eabc97432d7be9faedddf045ca7ad8fc/config.json
Model config LlamaConfig {
  "_name_or_path": "meta-llama/Llama-3.2-3B-Instruct",
  "architectures": [
    "LlamaForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "bos_token_id": 128000,
  "eos_token_id": [
    128001,
    128008,
    128009
  ],
  "head_dim": 128,
  "hidden_act": "silu",
  "hidden_size": 3072,
  "initializer_range": 0.02,
  "intermediate_size": 8192,
  "max_position_embeddings": 131072,
  "mlp_bias": false,
  "model_type": "llama",
  "num_attention_heads": 24,
  "num_hidden_layers": 28,
  "num_key_value_heads": 8,
  "pretraining_tp": 1,
  "rms_norm_eps": 1e-05,
  "rope_scaling": {
    "factor": 32.0,
    "high_freq_factor": 4.0,
    "low_freq_factor": 1.0,
    "original_max_position_embeddings": 8192,
    "rope_type": "llama3"
  },
  "rope_theta": 500000.0,
  "ti

## 12 Load PEFT Model

In [ ]:
torch.cuda.empty_cache()

In [ ]:
peft_path = save_path + '/LoRA'
peft_path

'./model/LoRA'

In [ ]:
peft_model = PeftModel.from_pretrained(model, peft_path)

## 13 Reload & Recheck Base Model

In [ ]:
model = load_model(model_name, base = False)
model

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--unsloth--Llama-3.2-3B-Instruct/snapshots/bc836a93eabc97432d7be9faedddf045ca7ad8fc/config.json
Model config LlamaConfig {
  "_name_or_path": "unsloth/Llama-3.2-3B-Instruct",
  "architectures": [
    "LlamaForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "bos_token_id": 128000,
  "eos_token_id": [
    128001,
    128008,
    128009
  ],
  "head_dim": 128,
  "hidden_act": "silu",
  "hidden_size": 3072,
  "initializer_range": 0.02,
  "intermediate_size": 8192,
  "max_position_embeddings": 131072,
  "mlp_bias": false,
  "model_type": "llama",
  "num_attention_heads": 24,
  "num_hidden_layers": 28,
  "num_key_value_heads": 8,
  "pretraining_tp": 1,
  "rms_norm_eps": 1e-05,
  "rope_scaling": {
    "factor": 32.0,
    "high_freq_factor": 4.0,
    "low_freq_factor": 1.0,
    "original_max_position_embeddings": 8192,
    "rope_type": "llama3"
  },
  "rope_theta": 500000.0,
  "tie_w

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 3072)
    (layers): ModuleList(
      (0-27): 28 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear4bit(in_features=3072, out_features=3072, bias=False)
          (k_proj): Linear4bit(in_features=3072, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=3072, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=3072, out_features=3072, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=3072, out_features=8192, bias=False)
          (up_proj): Linear4bit(in_features=3072, out_features=8192, bias=False)
          (down_proj): Linear4bit(in_features=8192, out_features=3072, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((3072,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((3072,), eps=1e

In [ ]:
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
trainable_percentage = (trainable_params / total_params) * 100

print('Total parameters :', total_params)
print('Trainable parameters :', trainable_params)
print('Trainable percentage: {:.2f}%'.format(trainable_percentage))

Total parameters : 1803463680
Trainable parameters : 394177536
Trainable percentage: 21.86%


In [ ]:
peft_model

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(128256, 3072)
        (layers): ModuleList(
          (0-27): 28 x LlamaDecoderLayer(
            (self_attn): LlamaSdpaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=3072, out_features=3072, bias=False)
                (lora_dropout): ModuleDict(
                  (LoRA): Dropout(p=0.1, inplace=False)
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (LoRA): Linear(in_features=3072, out_features=64, bias=False)
                  (default): Linear(in_features=3072, out_features=64, bias=False)
                )
                (lora_B): ModuleDict(
                  (LoRA): Linear(in_features=64, out_features=3072, bias=False)
                  (default): Linear(in_features=64, out_features=3072, bias=False)


In [ ]:
total_params = sum(p.numel() for p in peft_model.parameters())
trainable_params = sum(p.numel() for p in peft_model.parameters() if p.requires_grad)
trainable_percentage = (trainable_params / total_params) * 100

print('Total parameters :', total_params)
print('Trainable parameters :', trainable_params)
print('Trainable percentage: {:.2f}%'.format(trainable_percentage))

Total parameters : 1997974528
Trainable parameters : 0
Trainable percentage: 0.00%


## 13 Pre Test & Post Test

In [ ]:
def pre_assistant(prompt):
  messages = [
    {'role' : 'human', 'content' : prompt},
  ]
  inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True,
    return_tensors = 'pt',
    truncation = True,
    padding = True,
  ).to('cuda')
  generation_config = GenerationConfig(
    do_sample = True,
    top_k = 1,
    temperature = 0.1,
    max_new_tokens = 1024,
    pad_token_id = tokenizer.eos_token_id
  )
  outputs = model.generate(
    input_ids = inputs,
    generation_config = generation_config
  )
  return tokenizer.decode(outputs[0])#, skip_special_tokens = True)

In [ ]:
def post_assistant(prompt):
  messages = [
    {'role' : 'human', 'content' : prompt},
  ]
  inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True,
    return_tensors = 'pt',
    truncation = True,
    padding = True,
  ).to('cuda')
  generation_config = GenerationConfig(
    do_sample = True,
    top_k = 1,
    temperature = 0.1,
    max_new_tokens = 1024,
    pad_token_id = tokenizer.eos_token_id
  )
  outputs = peft_model.generate(
    input_ids = inputs,
    generation_config = generation_config
  )
  return tokenizer.decode(outputs[0])#, skip_special_tokens = True)

In [ ]:
def print_side_by_side(pre_text, post_text, width = 50):
  pre_wrapped = textwrap.wrap(pre_text, width)
  post_wrapped = textwrap.wrap(post_text, width)

  print('PRE-TEST'.center(width), ' | ', 'POST-TEST'.center(width))
  print(
    str(sum(p.numel() for p in model.parameters())).center(width),
    '|',
    str(sum(p.numel() for p in peft_model.parameters())).center(width)
  )
  print('=' * width, '|', '=' * width)

  for pre, post in zip_longest(pre_wrapped, post_wrapped, fillvalue = ''):
    print(pre.ljust(width), ' | ', post.ljust(width))

In [ ]:
prompt = dataset[randint(0, len(dataset))]['conversations'][0]['value']
pre_text = pre_assistant(prompt)
post_text = post_assistant(prompt)
print_side_by_side(pre_text, post_text)

                     PRE-TEST                       |                      POST-TEST                     
                    1803463680                     |                     1997974528                    
================================================== | ==================================================
<|begin_of_text|><|start_header_id|>system<|end_he  |  <|begin_of_text|><|start_header_id|>system<|end_he
ader_id|>  Cutting Knowledge Date: December 2023    |  ader_id|>  Cutting Knowledge Date: December 2023  
Today Date: 16 Nov 2024  <|eot_id|><|start_header_  |  Today Date: 16 Nov 2024  <|eot_id|><|start_header_
id|>human<|end_header_id|>  How can students learn  |  id|>human<|end_header_id|>  How can students learn
about the real-life applications of chemistry in    |  about the real-life applications of chemistry in  
everyday life despite limited exposure to           |  everyday life despite limited exposure to         
chemistry beyond the classroom?<|eot_id|><|start_h

In [ ]:
prompt = dataset[randint(0, len(dataset))]['conversations'][0]['value']
pre_text = pre_assistant(prompt)
post_text = post_assistant(prompt)
print_side_by_side(pre_text, post_text)

                     PRE-TEST                       |                      POST-TEST                     
                    1803463680                     |                     1997974528                    
================================================== | ==================================================
<|begin_of_text|><|start_header_id|>system<|end_he  |  <|begin_of_text|><|start_header_id|>system<|end_he
ader_id|>  Cutting Knowledge Date: December 2023    |  ader_id|>  Cutting Knowledge Date: December 2023  
Today Date: 16 Nov 2024  <|eot_id|><|start_header_  |  Today Date: 16 Nov 2024  <|eot_id|><|start_header_
id|>human<|end_header_id|>  What is the function    |  id|>human<|end_header_id|>  What is the function  
of the three tiny bones in the middle ear and how   |  of the three tiny bones in the middle ear and how 
do they contribute to the process of hearing?<|eot  |  do they contribute to the process of hearing?<|eot
_id|><|start_header_id|>assistant<|end_header_id|>

In [ ]:
prompt = dataset[randint(0, len(dataset))]['conversations'][0]['value']
pre_text = pre_assistant(prompt)
post_text = post_assistant(prompt)
print_side_by_side(pre_text, post_text)

                     PRE-TEST                       |                      POST-TEST                     
                    1803463680                     |                     1997974528                    
================================================== | ==================================================
<|begin_of_text|><|start_header_id|>system<|end_he  |  <|begin_of_text|><|start_header_id|>system<|end_he
ader_id|>  Cutting Knowledge Date: December 2023    |  ader_id|>  Cutting Knowledge Date: December 2023  
Today Date: 16 Nov 2024  <|eot_id|><|start_header_  |  Today Date: 16 Nov 2024  <|eot_id|><|start_header_
id|>human<|end_header_id|>  Solve the following     |  id|>human<|end_header_id|>  Solve the following   
math problem step-by-step. Simplify your answer as  |  math problem step-by-step. Simplify your answer as
much as possible. Present your final answer as      |  much as possible. Present your final answer as    
\boxed{Your Answer}. if the height of a cone is   

In [ ]:
prompt = dataset[randint(0, len(dataset))]['conversations'][0]['value']
pre_text = pre_assistant(prompt)
post_text = post_assistant(prompt)
print_side_by_side(pre_text, post_text)

                     PRE-TEST                       |                      POST-TEST                     
                    1803463680                     |                     1997974528                    
================================================== | ==================================================
<|begin_of_text|><|start_header_id|>system<|end_he  |  <|begin_of_text|><|start_header_id|>system<|end_he
ader_id|>  Cutting Knowledge Date: December 2023    |  ader_id|>  Cutting Knowledge Date: December 2023  
Today Date: 16 Nov 2024  <|eot_id|><|start_header_  |  Today Date: 16 Nov 2024  <|eot_id|><|start_header_
id|>human<|end_header_id|>  What were the social    |  id|>human<|end_header_id|>  What were the social  
and economic impacts of the Industrial Revolution   |  and economic impacts of the Industrial Revolution 
in 19th century England?<|eot_id|><|start_header_i  |  in 19th century England?<|eot_id|><|start_header_i
d|>assistant<|end_header_id|>  The Industrial     

In [ ]:
prompt = dataset[randint(0, len(dataset))]['conversations'][0]['value']
pre_text = pre_assistant(prompt)
post_text = post_assistant(prompt)
print_side_by_side(pre_text, post_text)

                     PRE-TEST                       |                      POST-TEST                     
                    1803463680                     |                     1997974528                    
================================================== | ==================================================
<|begin_of_text|><|start_header_id|>system<|end_he  |  <|begin_of_text|><|start_header_id|>system<|end_he
ader_id|>  Cutting Knowledge Date: December 2023    |  ader_id|>  Cutting Knowledge Date: December 2023  
Today Date: 16 Nov 2024  <|eot_id|><|start_header_  |  Today Date: 16 Nov 2024  <|eot_id|><|start_header_
id|>human<|end_header_id|>  How do viral            |  id|>human<|end_header_id|>  How do viral          
infections affect vulnerable populations such as    |  infections affect vulnerable populations such as  
infants, elderly and immunocompromised              |  infants, elderly and immunocompromised            
individuals, and what are the most effective      